In [1]:
# Load the CSV data into a DataFrame

import pandas as pd
df = pd.read_csv('feature_data3.csv')

# Convert the DataFrame to a NumPy array (matrix)
df_data = df.values

df_data = pd.DataFrame(df_data)

print(df_data)

           0         1          2           3           4           5   \
0    0.324081  0.113919  25.257430  221.713679  114.437756  254.385534   
1    0.891822  0.182954  11.288263   61.700028   61.073726   74.330344   
2    0.720290  0.183258  34.497578  188.245972  178.474088  267.226496   
3    0.456011  0.197963  36.007521  181.890080  114.354711  213.114992   
4    0.470130  0.136238  11.214203   82.313115   50.159745   91.005494   
..        ...       ...        ...         ...         ...         ...   
124  0.882767  0.241570  41.884166  173.383039  297.544955  212.150890   
125  0.434268  0.222052  40.629555  182.973112  155.012903  140.292551   
126  0.728373  0.234325  59.607648  254.380739  374.833296  216.187419   
127  0.923890  0.250879  53.087549  211.606485  346.255686  232.137890   
128  0.285109  0.063644  29.698362  466.631411  236.304888  124.679589   

             6           7              8           9   ...             90  \
0    227.002203  257.841036  2409

In [2]:
# import csv of lable
df = pd.read_csv('/home/tianyu/Desktop/data_base/labels.csv')

df_label = df['label'].values
df_label_num = []

for item in df_label:
    if item == 'M':
        df_label_num.append(1)
    else :
        df_label_num.append(0)
        
print(df_label_num)


[1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1]


In [14]:
# random forest k fold for old data and then test data

import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
group_models = []
group_num = 0
feature_importance = np.zeros(100)
X1 = df_data.iloc[:70]
for i in range(100):
    kf = KFold(n_splits=5, random_state=i+1, shuffle=True)

    X = df_data.iloc[0:70]
    y = df_label_num

    X = np.array(X)
    y = np.array(y)

    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf = RandomForestClassifier(n_estimators=20, random_state=i+1)
        clf.fit(X_train, y_train)
        
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        if accuracy >= 0.6:
            group_models.append(clf)
            group_num += 1
            
            importances = clf.feature_importances_
            for feature, importance in zip(X1.columns, importances):
                feature_importance[feature] += importance
    
    print(i)
    
print(group_num)
print(feature_importance)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
421
[ 6.28816059  2.95754861  4.09508398  7.72775541  4.11079921  9.22523435
  5.01493944  7.17139152  3.70212085  5.52090664  3.93110871  5.29756582
  2.77123465  4.29653427  2.60799207  2.55337241 20.00240001  3.14393322
  2.50041053  4.95316794  2.83784731  2.8887169   3.53446745  2.96780221
  5.13816561  3.69172179  3.06023738  3.00452649  3.89906886 19.03576483
  2.33851019  2.23063369  2.41800389  2.61000748  2.77468805  2.35061221
  3.17556122  2.87718454  6.28044306  2.28140218  2.7740788   2.06102161
  2.9451944   2.52550087 13.69001624  6.59931466  2.33973861  3.4596972
  2.36557078  2.94826843  2.55674993  2.12552849  2.91318118  3.00178608
  5.07781301  4.61483807  2.58133842  2.84574345  

In [15]:
# get test result through test data

import numpy as np
from scipy import stats
from sklearn.metrics import accuracy_score

X_task = df_data.iloc[70:100]
result_list = []

for model in group_models:
    result = model.predict(X_task)
    result_list.append(result)
    print(result)

vote_result = stats.mode(result_list, axis=0).mode.flatten()

print(f'final result is : \n {vote_result}')
print(df_label_num[70:100])

accuracy = accuracy_score(df_label_num[70:100], vote_result)
print(accuracy)

[1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 0]
[1 0 1 0 0 1 0 1 0 1 0 0 0 0 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0]
[1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 1 1 0 0 0 0 0]
[1 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0]
[1 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0]
[1 0 1 1 0 1 1 0 0 1 0 0 0 1 1 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0]
[1 1 1 1 0 1 0 0 1 1 0 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 0 0 1 1 0 0 0 0 1 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1 0 0 0 1 0 1 0 0 1 1]
[1 1 1 1 0 1 0 0 0 1 0 1 0 1 1 0 1 1 1 1 1 1 0 1 1 0 0 0 0 0]
[1 1 1 0 1 1 0 0 1 1 0 1 0 0 1 1 0 1 1 1 1 1 0 1 1 0 1 0 0 0]
[1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0]
[1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0]
[1 0 1 0

[1 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 1 1 1 1 1 0 0 1 1 0 0 0 1 0]
[1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0]
[1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0]
[1 0 1 0 0 1 0 0 1 1 0 0 0 1 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 1]
[1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1 0 1 0 1 1 1 1 0 1 0]
[1 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0]
[1 0 1 1 0 1 0 1 0 1 0 1 0 0 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0]
[1 0 1 0 0 1 0 0 0 0 0 1 0 1 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 1 0 1 0 0 0 1 0 1 0 0 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0]
[1 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 0 0 1 1 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 1 0 1 0]
[1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1 0 0 0 1 0 1 0 0 1 0]
[1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 1 1 0 0 0 1 1 0 0 0 1 0]
[1 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 1 0]
[1 0 1 1 0 1 0 1 0 0 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1 1 0 0 0 0]
[1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0]
[1 0 1 1

/tmp/ipykernel_19700/2069271138.py:15: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  vote_result = stats.mode(result_list, axis=0).mode.flatten()


In [10]:
# random forest k fold for old data and then test data

import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import lightgbm as lgb
group_models = []
group_num = 0

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',  
    'metric': 'binary_logloss',  
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

for i in range(100):
    kf = KFold(n_splits=5, random_state=i+1, shuffle=True)

    X = df_data.iloc[0:70]
    y = df_label_num

    X = np.array(X)
    y = np.array(y)

    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]\
        
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    
        gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=20
                    )
    
        y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

        y_pred = np.where(y_pred > 0.5, 1, 0)
        accuracy = accuracy_score(y_test, y_pred)
        if accuracy >= 0.6:
            group_models.append(clf)
            group_num += 1
    
    print(i)
    
print(group_num)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000076 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000047 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000014 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000014 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000013 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000014 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000014 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

In [11]:
# get test result through test data

import numpy as np
from scipy import stats
from sklearn.metrics import accuracy_score

X_task = df_data.iloc[70:100]
result_list = []

for model in group_models:
    result = model.predict(X_task)
    result_list.append(result)
    print(result)

vote_result = stats.mode(result_list, axis=0).mode.flatten()

print(f'final result is : \n {vote_result}')
print(df_label_num[70:100])

accuracy = accuracy_score(df_label_num[70:100], vote_result)
print(accuracy)

[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0

[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0

/tmp/ipykernel_19700/2069271138.py:15: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  vote_result = stats.mode(result_list, axis=0).mode.flatten()


In [12]:
from catboost import CatBoostClassifier

group_models = []
group_num = 0

for i in range(100):
    kf = KFold(n_splits=5, random_state=i+1, shuffle=True)

    X = df_data.iloc[0:70]
    y = df_label_num

    X = np.array(X)
    y = np.array(y)

    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]\
        
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    
        gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=20
                    )
    
        y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

        y_pred = np.where(y_pred > 0.5, 1, 0)
        accuracy = accuracy_score(y_test, y_pred)
        if accuracy >= 0.6:
            group_models.append(clf)
            group_num += 1
    
    print(i)
    
print(group_num)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000077 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

14
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000046 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightG

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000015 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000014 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000015 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000048 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000074 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

In [13]:
# get test result through test data

import numpy as np
from scipy import stats
from sklearn.metrics import accuracy_score

X_task = df_data.iloc[70:100]
result_list = []

for model in group_models:
    result = model.predict(X_task)
    result_list.append(result)
    print(result)

vote_result = stats.mode(result_list, axis=0).mode.flatten()

print(f'final result is : \n {vote_result}')
print(df_label_num[70:100])

accuracy = accuracy_score(df_label_num[70:100], vote_result)
print(accuracy)

[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0

[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0]
[1 0 1 0

/tmp/ipykernel_19700/2069271138.py:15: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  vote_result = stats.mode(result_list, axis=0).mode.flatten()
